Ref : https://www.sbert.net/docs/training/overview.html#creating-networks-from-scratch


In [ ]:
pip install -U sentence-transformers

In [1]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

model = SentenceTransformer('distilbert-base-nli-mean-tokens')

ModuleNotFoundError: ignored


## Data Preprocessing

In [ ]:
import pandas as pd

df_2 = pd.read_csv("./2_man_ann_sb.csv",  index_col= None)
df_2 = df_2.dropna(subset = ['tweet_text'])

df_1 = pd.read_csv("./mann_ann_sb.csv", index_col= None)

df_3 = pd.read_csv("./3_man_ann_sb_full_1.csv", index_col = None)
df_3 = df_3.dropna(subset = ['tweet_text'])

df_raw = df_1.append(df_2).append(df_3) # using batch 1 and batch 2 for training

print(df_raw.shape)

In [ ]:
# Remove ads marked as below because they are not available in ad annotations file although tweets mention them
# commercials, joe biden, pizzahut, joe bieden, michael bloomberg, mike bloomberg, scientology...
ads_remove = ['commercials', 'joe biden', 'pizzahut', 'joe bieden', 'michael bloomberg', 'mike bloomberg', 
              'scientology','papa johns',  'bakari',  'secret', 'dashlane', 'bernie the peoples perfume',
              'ram trucks', 'golden gronks', "bush's best", 'ragged old flag', 'patience', 'guitar hero',
              'disney mulan']

# ads_rename = ['nfl100', 'tide']
# rename ads with spelling faults while manually adding the annotation 

df_raw['ad_manual_adjusted'] = df_raw['ad_manual_adjusted'].apply(lambda x: x.lower())
df_raw.loc[df_raw.ad_manual_adjusted == "discover card  no we don‚äôt charge annual fees", 
       "ad_manual_adjusted"] = "discover card  no we don’t charge annual fees"
df_raw.loc[df_raw.ad_manual_adjusted == "doritos the cool ranch", 
       "ad_manual_adjusted"] = "doritos  the cool ranch"
df_raw.loc[df_raw.ad_manual_adjusted == "discover card yes we're accepted", 
       "ad_manual_adjusted"] =  "discover card  yes we’re accepted"
df_raw.loc[df_raw.ad_manual_adjusted == "discover card yes we’re accepted", 
       "ad_manual_adjusted"] =  "discover card  yes we’re accepted"
df_raw.loc[df_raw.ad_manual_adjusted == "discover card  yes we're accepted", 
       "ad_manual_adjusted"] =  "discover card  yes we’re accepted"
df_raw.loc[df_raw.ad_manual_adjusted == "budweiser typical american", 
       "ad_manual_adjusted"] = "budweiser  typical american"
df_raw.loc[df_raw.ad_manual_adjusted == 'fox  halftime show  teaser_3',
            "ad_manual_adjusted"] = "fox  halftime show  teaser_1"
df_raw.loc[df_raw.ad_manual_adjusted == 'fox  halftime show  teaser_2',
            "ad_manual_adjusted"] = "fox  halftime show  teaser_1"
            
print(df_raw.shape)
df = pd.DataFrame()
removed_Data = pd.DataFrame()

for i,row  in df_raw.iterrows():
  if row['ad_manual_adjusted'] not in ads_remove:
    df = df.append(row)
  else:
    removed_Data = removed_Data.append(row)
print(df.shape)

In [ ]:
classType = 'sent_exploded' # binary or multi-class or sent-exploded

In [ ]:
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')

stop = stopwords.words('english')
# stop.append('superbowl')
# stop.append('super') 
# stop.append('bowl')

# remove for multi class since almost all ads have these words
if classType is not 'binary':
  stop.append('commercial')
  stop.append('ad')
  stop.append('commercials')
  stop.append('ads')
print(len(stop))

def removeMentions(text):

    textBeforeMention = text.partition("@")[0]
    textAfterMention = text.partition("@")[2]
    textAfterMention =  re.sub(r':', '', textAfterMention) #cadillac join the 31k
    tHandle = textAfterMention.partition(" ")[0].lower() #cadillac    
    text = textBeforeMention+ " " + textAfterMention  
    return text

def cleanTweet(strinp):
    strinp = re.sub(r'RT', "", strinp) # Remove RT
    strinp = strinp.lower()
    
    stop_removed_list = [word for word in strinp.split() if word not in (stop)]
    stop_removed = ' '.join([str(elem) for elem in stop_removed_list])    
    text = re.sub('https?://[A-Za-z0-9./]+', ' ', stop_removed) # Remove URLs
    text = removeMentions(text)
    text = re.sub('[^\x00-\x7F]+', ' ', text) # Remove non-ASCII chars.
    
    # remove punctuations except '-'
    punctuation = ['(', ')', '[',']','?', ':', ':', ',', '.', '!', '/', '"', "'", '@', '#', '&', '-', '_']
    text = "".join((char for char in text if char not in punctuation))
    text = re.sub('[^a-zA-Z]', ' ', text) # remove all other than alphabet chars 

#     text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text) # remove all single characters     
    stop_removed_l = [word for word in text.split() if word not in (stop)]
    stop_removed = ' '.join([str(elem) for elem in stop_removed_l]) 
    return stop_removed

print(cleanTweet("RT @cadillacabc: Joinrt the 31K james_bond") )

In [ ]:
df['text_clean'] = df['tweet_text'].apply(lambda x: cleanTweet(x))
df['ad_manual_adjusted'] = df['ad_manual_adjusted'].apply(lambda x: x.lower())
df['ad_related'] = df['ad_manual_adjusted'].apply(lambda ad: 0 if ad == 'none' else 1)

comma_filter = ~df['ad_manual_adjusted'].str.contains(',')
df = df[comma_filter]

In [ ]:
df_unique = df.drop_duplicates(subset = ['text_clean'])
df_with_dupes = df
df = df_unique

print(df_with_dupes.shape)
print(df.shape)

In [ ]:
ad_product_df = pd.read_csv('./SB_ad_annotations_product_category_modified.csv')
ad_product_df = ad_product_df.rename(columns = {'Ad Name': 'Ad_Name'}) # rename the column to remove space
ad_product_df = ad_product_df.dropna() # because the file has trailing empty rows, remove them
# remove fox half time show teaser_2 ad because its keywords is same as fox half time show teaser_2
print(ad_product_df.shape)
ad_product_df.drop(ad_product_df[ad_product_df['Ad_Name'] == 'FOX  Halftime Show  Teaser_2'].index, inplace = True) 
print(ad_product_df.shape)
ad_product_dict = dict()

ad_product_df['Product_modified'] = ad_product_df['Product_modified'].apply(lambda x: x.lower())
for i, row in ad_product_df.iterrows():
  ad_product_dict[row['Ad_Name'].lower()] = row['Product_modified'].lower()

ad_product_dict['none'] = 'none'
print(ad_product_dict)

df['product_modified'] = df['ad_manual_adjusted'].apply(lambda ad: ad_product_dict[ad])
df['product_modified'] = df['product_modified'].apply(lambda x: x.lower())

In [ ]:
# group the keywords of ads part of a product bucket and append these to the training sentences
ad_product_keywords_dict = ad_product_df.groupby('Product_modified')['Key Terms  Round 2'].agg(lambda x : x.sum() if x.dtype=='float64' else ' '.join(x))
# clean the ad key words - not removing duplicate words here - TODO
for ad_bucket in ad_product_keywords_dict.keys():
  ad_product_keywords_dict[ad_bucket] = cleanTweet(ad_product_keywords_dict[ad_bucket])
ad_product_df['product_modified_keywords'] = ad_product_df['Product_modified'].apply(lambda x: ad_product_keywords_dict[x])
ad_product_df.head(2)

# generate adname, ad keywords dict to use in sent exploding
ad_name_keywords_dict = dict()
for i, row in ad_product_df.iterrows():
  ad_name_keywords_dict[row['Ad_Name'].lower()] = cleanTweet(row['Key Terms  Round 2'])
ad_name_keywords_dict['none'] = 'none'
print(ad_name_keywords_dict)

ad_keywords_name_dict = dict()
for ad_name in ad_name_keywords_dict:
  keywords_temp = ad_name_keywords_dict[ad_name]
  ad_keywords_name_dict[keywords_temp] = ad_name
print(ad_keywords_name_dict)

ad_product_df['ad_name_keywords'] = ad_product_df['Ad_Name'].apply(lambda x: ad_name_keywords_dict[x.lower()])

In [ ]:
# use column name 'ad_manual_adjusted' of df to get ad_name
def get_ad_related_twts(df, removeCommas = True):
  df['ad_manual_adjusted'] = df['ad_manual_adjusted'].apply(lambda x: x.lower())
  ad_filter = df['ad_manual_adjusted']!= 'none'
  ad_related_twts = df[ad_filter]
  if removeCommas:
    ad_filter_1 = ~ad_related_twts['ad_manual_adjusted'].str.contains(',')
    ad_related_twts = ad_related_twts[ad_filter_1]
  return ad_related_twts

def getAdTweets(ad_related_twts, ad):
  return ad_related_twts[ad_related_twts.ad_manual_adjusted == ad].shape[0]

def get_ad_id_dict(ad_related_twts): 
  n_ad_related = ad_related_twts.shape[0]
  print("# ad related tweets: "+ str(n_ad_related))
  ads_annotated = ad_related_twts.ad_manual_adjusted.values
  adset = set(ads_annotated)
  print("unique ads:"+ str(len(adset)))
  ad_id_dict = {}
  i = 0
  for ad in adset : 
    if(getAdTweets(ad_related_twts, ad) >=2):
      ad_id_dict[ad] = i
      i = i+1
    else:
      print('ad with <2 samples: '+ str(ad))
  print(" No of ads with >=2 samples:"+ str(len(ad_id_dict)))
  ad_id_dict['none'] = len(ad_id_dict)
  print(ad_id_dict)
  return ad_id_dict

def convertAdNameToAdId(ad_id_dict, ad_name):
  if ad_name in ad_id_dict:
    return ad_id_dict[ad_name]
  else:
    return ad_id_dict['none']

ad_related_twts = get_ad_related_twts(df)
ad_id_dict = get_ad_id_dict(get_ad_related_twts(df))
n_unique_ads = len(ad_id_dict) # ad_id_dict has none as well, so minus 1 when using embeddings
df['ad_manual_adjusted_id'] = df['ad_manual_adjusted'].apply(lambda x: convertAdNameToAdId(ad_id_dict,x))
print(n_unique_ads)


#Cosine sim or embeddings

In [ ]:
# from sentence_transformers import util

# tweets = df['text_clean'].values
# ad_names = df['ad_manual_adjusted'].values
# ad_keywords = list(ad_name_keywords_dict.values())

# twt_embeddings = model.encode(tweets, convert_to_tensor = True)
# ad_embeddings = model.encode(ad_keywords, convert_to_tensor = True)

# #Compute cosine-similarities for each sentence with each other sentence
# cosine_scores = util.pytorch_cos_sim(twt_embeddings, ad_embeddings)

# #Find the pairs with the highest cosine similarity scores
# pairs = []
# for i in range(len(cosine_scores)-1):
#     max_ad_keywords = ''
#     max_score = 0
#     ad_sentBERT = ''
#     max_score_ad_id = 0
#     for j in range(0, len(ad_embeddings)):
#         if(cosine_scores[i][j] > max_score):
#           max_score = cosine_scores[i][j]
#           max_ad_keywords = ad_keywords[j]
#           ad_sentBERT = ad_keywords_name_dict[max_ad_keywords]
#           max_score_ad_id = j
#         # pairs.append({'index': [i, j], 'score': cosine_scores[i][j],'tweet': tweets[i],  'ad_sentBERT_keywords': curr_ad_keywords,'ad_sentBERT': ad_keywords_name_dict[curr_ad_keywords] })
#     pairs.append({'index': [i, max_score_ad_id], 'score': max_score,'tweet': tweets[i],  'ad_sentBERT_keywords': max_ad_keywords,'ad_sentBERT': ad_sentBERT })
# #Sort scores in decreasing order
# #pairs = sorted(pairs, key= lambda x: x['score'], reverse=True)

# for pair in pairs[0:10]:
#     i, j = pair['index']
#     print("{} \t\t {} \t\t Score: {:.4f}".format(tweets[i], ad_keywords[j], pair['score']))

In [ ]:
# df_pairs = pd.DataFrame(pairs, columns=['index', 'score', 'tweet', 'ad_sentBERT_keywords', 'ad_sentBERT'])

# df['ad_sentBERT'] = df_pairs['ad_sentBERT']
# df['score'] = df_pairs['score']
# df['match'] = df.apply(lambda row: 1 if row['ad_sentBERT'] == row['ad_manual_adjusted'] else 0, axis = 1)

# print(df.shape)
# print(df['match'].sum())
# print(df['match'].sum() * 100/ df.shape[0])
# df.to_csv('./sent_bert_results.csv')

**Train the model**

Loss Function - CosineSimilarityLoss

In [ ]:
# train test split for multi class classification
from sklearn.model_selection import train_test_split

n = df.shape[0]
print(n)
# sentences, test_sentences, labels, test_labels = train_test_split(df.text_clean.values, 
#                   df.ad_manual_adjusted.values, 
#                   random_state = 2018, 
#                    test_size = 0.2, stratify = df.ad_manual_adjusted.values)
sentences, test_sentences, labels, test_labels = train_test_split(df.text_clean.values, 
                  df.ad_manual_adjusted.values, 
                  random_state = 2018, 
                   test_size = 0.2)
train_size = len(sentences)
test_size = len(test_sentences)
print( "Train size: "+ str(train_size)+" test size:" + str(test_size))

In [ ]:
ad_names = list(ad_id_dict.keys())

def build_train_data(twt, label_true):
  examples = []
  for ad in ad_names:
      if label_true == ad:
        examples.append(InputExample(texts=[twt, ad_name_keywords_dict[label_true] ], label= 1.0))
      else:
        examples.append(InputExample(texts=[twt, ad_name_keywords_dict[label_true] ], label= 0.0))
  return examples

train_ex_nested = [ build_train_data(sentences[i], labels[i]) for i in range(0, len(sentences)) ]

train_examples = []
for row in train_ex_nested:
  train_examples.extend(row)

print(len(train_examples))


# Sentence Transformer Model

In [ ]:
# train_dataloader = DataLoader(train_examples, shuffle=True, batch_size = 200)
# train_loss = losses.CosineSimilarityLoss(model)
model = SentenceTransformer('nq-distilbert-base-v1')

# # model.fit(train_objectives=[(train_dataloader, train_loss)], epochs = 3, warmup_steps=100, evaluation_steps=500)
# model.fit(train_objectives=[(train_dataloader, train_loss)], epochs = 1)
# #           ,output_path=model_save_path)
# #start: 11:34 am
# # takes 3 hrs for 1000 input training samples i.e 61K exploded

# Cross Encoder Model

In [ ]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('cross-encoder/stsb-TinyBERT-L-4')

In [ ]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size = 200)
# train_loss = losses.CosineSimilarityLoss(model)
model.fit(train_dataloader)
# 23 sec for 1000 samples 

In [ ]:
model.save('./cross_enc_model')

In [ ]:
from sklearn. preprocessing import normalize
import numpy as np

ad_keywords = list(ad_name_keywords_dict.values())
ad_names = list(ad_name_keywords_dict.keys())

def get_most_sim_ad(test_twt):
  sims = []
  for ad_keyword in ad_keywords:
    sims.append(model.predict([test_twt, ad_keyword]))

  norm_sims = normalize(np.array(sims).reshape(1, -1))
  max_sim = max(norm_sims)
  max_ad_ind = np.argmax(norm_sims)
  max_ad = ad_names[max_ad_ind]
  # print(test_twt+":"+ max_ad)
  return max_ad

max_ad = get_most_sim_ad('nyxipuff alexa like tweet')

# pred_cross_enc_train = [get_most_sim_ad(train_twt) for train_twt in sentences[0: 500]]

pred_cross_enc = [get_most_sim_ad(test_twt) for test_twt in test_sentences]


# Evaluate model

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report, confusion_matrix

def getMetrics(true_labels_flat, pred_labels, averageType):
  print("Evaluating metrics as per '"+averageType+"' average type")
  f1score = f1_score(true_labels_flat, pred_labels, average= averageType) 
  print('f1_score:'+ str(f1score))

  prec = precision_score(true_labels_flat, pred_labels, average=averageType)
  print('Precision:'+ str(prec))

  acc = accuracy_score(true_labels_flat, pred_labels)
  print("Accuracy: "+ str(acc))

  recall = recall_score(true_labels_flat, pred_labels, average=averageType)
  print("recall: "+ str(recall))

  class_report = classification_report(true_labels_flat, pred_labels, output_dict = True)
  class_report_df = pd.DataFrame(class_report)
  print(class_report)
  class_report_df.to_csv("./class_report_cross_enc.csv")

  confusionmatrix = confusion_matrix(true_labels_flat, pred_labels)
  #print(confusionmatrix)
  # if averageType == 'binary':
    # tn, fp, fn, tp = confusionmatrix.ravel()
    # print("tp: "+ str(tp)+" tn: "+ str(tn)+" fp: "+ str(fp)+" fn: "+ str(fn))

if classType == 'binary':
  averageType = 'binary'
elif classType == 'sent_exploded':
  averageType = 'weighted' # no weighting for imbalance
else:
  averageType = 'weighted'
  # micro: Calculate metrics globally by counting the total true positives, false negatives and false positives.
  # macro: Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.
  # weighted : Calculate metrics for each label, and find their average weighted by support
  #  (the number of true instances for each label). This alters ‘macro’ to account for label imbalance; 
  # it can result in an F-score that is not between precision and recall.
classType = 'sent_exploded'
getMetrics(test_labels, pred_cross_enc, averageType)

Evaluating metrics as per 'weighted' average type
f1_score:0.010613120784794606
Precision:0.02832618025751073
Accuracy: 0.0068669527896995704
recall: 0.0068669527896995704
{'amazon echo  before alexa': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12}, 'amazon prime video  hunters': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, 'audi  let it go [t1]': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9}, 'avocados from mexico  the avocados from mexico shopping network': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5}, 'bud light seltzer  posty store  inside post s brain': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10}, 'budweiser  typical american': {'precision': 1.0, 'recall': 0.14285714285714285, 'f1-score': 0.25, 'support': 7}, 'cheetos  can t touch this': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 7}, 'coca-cola energy  show up': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarnin

In [ ]:
from sentence_transformers import evaluation
sentences1 = ['This list contains the first column. With your sentences', 'With your sentences', 'You want your model to evaluate on']
sentences2 = ['Sentences contains the other column', 'The evaluator matches sentences1[i] with sentences2[i]', 'Compute the cosine similarity and compares it to scores[i]']
scores = [0.3, 0.6, 0.2]

evaluator = evaluation.EmbeddingSimilarityEvaluator(sentences1, sentences2, scores)
#evaluator = EmbeddingSimilarityEvaluator.from_input_examples(sts_reader.get_examples('sts-dev.csv'))


In [ ]:
# ax+by+cz 

# a1x+b1y+c1z 

# Actual similarity = 0.3

# BERT - 3000 words

# If similarity is computed directly, the ad vector might not be in the vicinty of ad key word vector, 
# hence giving less similarity score

# Issues:
# RHS - Not a full sentence: 

# todo:
# For fine Tuning have similarity scores as 0 if not ad related , 1 if ad related
